In [ ]:
def subdivizija(G):
    """Makes a subdivision of a given graph"""
    H = G.copy()
    H.subdivide_edges(G.edges(), 1)
    return H

def poisci_pakirno_stevilo(G, max_colors, min_packing_number):
    # poišče pakirno število grafa G, če je le to >= min_packing_number. Št. barv <= max_colors
    # vrne trojico: graf G, pakirno število grafa, slovar vozlišč s pripadajočimi barvami
    
    V = G.vertices()
    D = G.distance_matrix()
    
    # ustvarimo CLP
    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) # x_v,i -> vozlišče v je pobarvano z barvo i
    z = p.new_variable(nonnegative=True) # šteje koliko barv smo porabili (pakirno število)
    
    # minimiziramo število uporabljenih barv
    p.set_objective(z[0])
    
    # omejitve:
    # 1) vsako vozlišče je pobarvano z natanko eno barvo
    for v in V:
        p.add_constraint(sum(x[v, i] for i in range(1, max_colors + 1)) == 1)
    
    # 2) če sta vozlišči oddaljeni <i+1 sta pobarvani z različno barvo
    for v in V:
        for u in V:
            for i in range(1, max_colors + 1): # range(1, D[v][u] + 1)
                if (D[v][u] <= i) and (v != u):
                    p.add_constraint(x[v, i] + x[u, i] <= 1)
    
    # 3) barva vsakega vozlišča ni večja od pakirnega števila
    for v in V:
        for i in range(1, max_colors + 1):
            p.add_constraint(i * x[v, i] <= z[0])                

    # reši CLP
    p.solve()
    [x_sol, z_sol] = p.get_values(x, z[0]) 

    # vrnemo le grafe s pakirnim številom >= min_packing_number
    if z_sol >= min_packing_number:
        # ustvari slovar, ključi so vozlišča, vrednosti so barve vozlišč
        barve_vozlisc = {v: next(i for i in range(1, max_colors + 1) if x_sol[v, i] > 0.5) for v in V}
        return (G, z_sol, barve_vozlisc)
    else:
        return



In [ ]:
def connected_subgraphs(G):
    """
    Generate all connected subgraphs of a given graph G.

    Parameters:
        G (Graph): The input graph.

    Returns:
        List[Graph]: A list of all connected subgraphs of G.
    """
    vertices = G.vertices()
    n = len(vertices)
    connected_subgraphs = []
    
    # Iterate over all subsets of vertices (non-empty)
    for subset_mask in range(1, 2**n):  # 1 to 2^n - 1 (all non-empty subsets)
        subset = [vertices[i] for i in range(n) if (subset_mask & (1 << i))]
        subgraph = G.subgraph(subset)
        
        # Check if the induced subgraph is connected
        if subgraph.is_connected():
            connected_subgraphs.append(subgraph)
    
    return connected_subgraphs


In [ ]:
def random_cubic_graphs(zacetno_stevilo_vozlisc, koncno_stevilo_vozlisc, zeljeno_barvanje):  
    """Generates cubic graphs on even verticies betwen zacetno_stevilo_vozlisc and koncno_stevilo_vozlisc 
    and checks if the packing coloring is greater or equal to zeljeno_barvanje"""  
    if zacetno_stevilo_vozlisc % 2 != 0:
        print('Zacetno stevilo vozlišč mora biti sodo')# Kubični grafi so lahko samo tisi, ki majo sodo število vozlišč
        return None
    n = zacetno_stevilo_vozlisc
    list_of_graphs = []
    invalid_steps = 0
    
    while n <= koncno_stevilo_vozlisc:
        while invalid_steps < 10: # To povečava ko povečava število vozlišč
            G = graphs.RandomRegular(3, n)
            stevilo = invalid_steps
            if G.is_bipartite():
                continue
            for graph in list_of_graphs:    
                if G.is_isomorphic(graph) == True:
                    invalid_steps += 1
                    break
            if stevilo != invalid_steps:
                continue
            new_G = subdivizija(G)
            (a,coloring,b) = poisci_pakirno_stevilo(new_G, 7, 3)
            list_of_graphs.append(G)
            if coloring >= zeljeno_barvanje:
                return G
        n += 2
    print('Končano')